In [23]:
toyData=open('toy_data', 'r')

In [5]:
pd.DataFrame(toyData.read())

ValueError: DataFrame constructor not properly called!

In [55]:
import pandas as pd
import numpy as np

In [24]:
data=toyData.read()

In [7]:
for item in data:
    print(item)

In [15]:
toyData.readline()

''

In [18]:
import ast
import json

In [28]:
df = pd.DataFrame(ast.literal_eval(data))

In [30]:
df['comment'].apply(lambda x: x.replace("<[^>]*>", ""))

0      How do you all monitor air quality?<p>I person...
1      Any idea how good the Awair monitors are? I re...
2      The South Coast (California) Air Quality Manag...
3      Anyone want to review this table and suggest a...
4      Based on my reading of the table, the PurpleAi...
                             ...                        
100    <a href="https:&#x2F;&#x2F;en.wikipedia.org&#x...
101    I&#x27;m confused - is IQ a reliable way to me...
102    whatever skills IQ tests measure these do corr...
103    I thought IQ was not a reliable measure of any...
104    I wouldn&#x27;t call IQ a completely useless y...
Name: comment, Length: 105, dtype: object

In [40]:
htmlcomm=df.iloc[100, 1]

In [35]:
from html.parser import HTMLParser

In [37]:
parser= HTMLParser()

In [38]:
parser.feed(htmlcomm)

In [42]:
parser.handle_data(htmlcomm)

In [41]:
htmlcomm

'<a href="https:&#x2F;&#x2F;en.wikipedia.org&#x2F;wiki&#x2F;Stanford_marshmallow_experiment" rel="nofollow">https:&#x2F;&#x2F;en.wikipedia.org&#x2F;wiki&#x2F;Stanford_marshmallow_experimen...</a><p>&gt; The replication suggested that economic background, rather than willpower, explained the other half<p>Yeah<p>And 250k&#x2F;y &quot;poor&quot;<p><a href="https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;the-self-pity-of-the-harvard-poor&#x2F;63755&#x2F;" rel="nofollow">https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;the-sel...</a><p><a href="https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;in-defense-of-the-self-pitying-wealthy-poor&#x2F;63777&#x2F;" rel="nofollow">https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;in-defe...</a>'

In [43]:
import re
re.sub("<[^>]*>", "", htmlcomm)

'https:&#x2F;&#x2F;en.wikipedia.org&#x2F;wiki&#x2F;Stanford_marshmallow_experimen...&gt; The replication suggested that economic background, rather than willpower, explained the other halfYeahAnd 250k&#x2F;y &quot;poor&quot;https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;the-sel...https:&#x2F;&#x2F;www.theatlantic.com&#x2F;national&#x2F;archive&#x2F;2010&#x2F;09&#x2F;in-defe...'

In [49]:
df['cleanComment']=df['comment'].apply(lambda html: BeautifulSoup(html).get_text())

In [47]:
from bs4 import BeautifulSoup

In [52]:
print(df['cleanComment'].iloc[100])

https://en.wikipedia.org/wiki/Stanford_marshmallow_experimen...> The replication suggested that economic background, rather than willpower, explained the other halfYeahAnd 250k/y "poor"https://www.theatlantic.com/national/archive/2010/09/the-sel...https://www.theatlantic.com/national/archive/2010/09/in-defe...


In [54]:
def removeLinks(data):
    if 'href' in data: 
        return np.NaN
    else:
        return data

In [61]:
df['cleanComment']=df['comment'].apply(removeLinks).dropna().apply(lambda x: BeautifulSoup(x).get_text())

In [58]:
df.shape

(105, 3)

In [64]:
df.dropna(inplace=True)

In [66]:
df.shape

(81, 3)

In [67]:
df.head()

,user,comment,cleanComment
0,Scene_Cast2,How do you all monitor air quality?<p>I person...,How do you all monitor air quality?I personall...
3,PowerfulWizard,Anyone want to review this table and suggest a...,Anyone want to review this table and suggest a...
4,bearforcenine,"Based on my reading of the table, the PurpleAi...","Based on my reading of the table, the PurpleAi..."
6,9dl,&gt;*Field R2<p>Under the table is definition ...,>*Field R2Under the table is definition of R^2...
7,bearforcenine,Isn&#x27;t the table sorted alphabetically?,Isn't the table sorted alphabetically?


In [71]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [72]:
analyser = SentimentIntensityAnalyzer()

In [84]:
posScore=[]
neuScore=[]
comScore=[]
negScore=[]
for comment in df['cleanComment']:
    neg, neu, pos, com = analyser.polarity_scores(comment).values()
    negScore.append(float(neg))
    neuScore.append(float(neu))
    posScore.append(float(pos))
    comScore.append(float(com))
print(np.mean(negScore))
print(np.mean(neuScore))
print(np.mean(posScore))
print(np.mean(comScore))

0.09486419753086421
0.8171111111111112
0.08798765432098764
0.039937037037037035


In [75]:
df['user'].value_counts()

glofish            6
INGELRII           2
OnlineGladiator    2
gnode              2
TrainedMonkey      2
                  ..
acd                1
avip               1
bryanmgreen        1
ErikAugust         1
jefftk             1
Name: user, Length: 63, dtype: int64

In [85]:
def get_scores(data):
    '''Returns the mean scores of negative, neutral, positive or combined for 
    a given list of data 
    would make sense to be grouped by user or by article'''
    posScore=[]
    neuScore=[]
    comScore=[]
    negScore=[]
    for comment in data:
        neg, neu, pos, com = analyser.polarity_scores(comment).values()
        negScore.append(float(neg))
        neuScore.append(float(neu))
        posScore.append(float(pos))
        comScore.append(float(com))
    return {'neg': np.mean(negScore), 'neu': np.mean(neuScore), 'pos':np.mean(posScore), 'com': np.mean(comScore)}

In [ ]:
def negativity_func(text):
    return analyser.polarity_scores(text)['neg']

In [86]:
analyser.polarity_scores('this is a line of text')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}